In [ ]:
# default_exp feature_testing

# Testing features

> Testing the impact on features, generated in `preprocessing`, on the model performance

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from loguru import logger
from fastai.tabular.all import *
from ashrae import loading, preprocessing, inspection
from sklearn import linear_model, tree, model_selection, ensemble
import tqdm
import plotly.express as px

import pandas as pd
import ipywidgets as widgets

In [ ]:
pd.options.plotting.backend = "plotly"

## Loading

In [ ]:
%%time
ashrae_data = loading.load_all()

## Executing the pre-processing

In [ ]:
%%time
tfms_config = {
#     'fix_bid_363':{},
#     'fix_bid_1099':{'threshold': 10.},
#     'remove_bad_meter0_readings_of_first_141days': {},
#     'remove_not_summer_0s_meter_2_and_3': {},
#     'remove_0s_meter0': {},
#     'remove_outliers':{'f':10,'dep_var':'meter_reading'},
#     'remove_imputed_weeks':{'dep_var':'meter_reading'},
#     'add_dep_var_stats':{},
    'add_time_features':{},
    'add_weather_features':{'fix_time_offset':True,
                            'add_na_indicators':True,
                            'impute_nas':True},
    'add_building_features':{},
#     'add_onehot_encoded':{},
}

df, df_test, var_names = preprocessing.preprocess_all(ashrae_data, tfms_config)

## Testing if certain features improve the score beyond the baseline

Training to get a basic idea if the added features do have any benefit

In [ ]:
%%time
#df = load_df(data_path/'X.parquet')

%time
#df_test_p = load_df(data_path/'X_test.parquet')

%time
#var_names = load_var_names(data_path/'var_names.pckl')

In [ ]:
#export
def get_tabular_object(df:pd.DataFrame, var_names:dict,
                       splits=None, procs:list=None):
    if procs is None: procs = []
    return TabularPandas(df.copy(), procs,
                         var_names['cats'], var_names['conts'],
                         y_names=var_names['dep_var'],
                         splits=splits)

SPLIT_PARAMS = dict(
    train_frac = .8,
    split_kind = 'time_split_day',
)


def train_predict(df:pd.DataFrame, var_names:dict,
                  model, model_params:dict=None, n_rep:int=3,
                  n_samples_train:int=10000,
                  n_samples_valid:int=10000,
                  procs:list=[Categorify, FillMissing, Normalize],
                  split_params:dict=None):

    split_params = SPLIT_PARAMS if split_params is None else split_params
    y_col = var_names['dep_var']
    score_vals = []
    model_params = {} if model_params is None else model_params

    to = get_tabular_object(df, var_names, procs=procs)

    for i in tqdm.tqdm(range(n_rep), total=n_rep, desc='Repetition'):
        m = model(**model_params)
        splits = preprocessing.split_dataset(df, **split_params)

        mask = to.xs.index.isin(splits[0])

        _X = to.xs.loc[~mask, :].iloc[:n_samples_train]
        _y = to.ys.loc[~mask, y_col].iloc[:n_samples_train]
        m.fit(_X.values, _y.values)

        _X = to.xs.loc[mask, :].iloc[:n_samples_valid]
        _y = to.ys.loc[mask, y_col].iloc[:n_samples_valid]
        pred = m.predict(_X.values)
        s = torch.sqrt(F.mse_loss(tensor(pred), tensor(_y.values))).item()
        score_vals.append({'iter': i, 'rmse loss': s})

    return pd.DataFrame(score_vals)

In [ ]:
split_params = dict(
    #split_kind = 'random',
    #split_kind = 'time',
    #split_kind = 'fix_time',
    split_kind = 'time_split_day',
    t_train = None,
    train_frac = .8,
)

In [ ]:
model_params = {'n_estimators': 40, 'max_features': 'sqrt',
          'n_jobs': -1, 'min_samples_leaf':25}
model = ensemble.RandomForestRegressor
# params = {}
# model = linear_model.LinearRegression
n_rep = 21
n_samples_train = 100000
n_samples_valid = 100000

The following is not always necessary. Sensible in the case of a linear model to remove categorical values which are not onehot encoded

In [ ]:
# to_remove = {'cats':['building_id', 'meter'], 'conts': []}

# for k in ['cats', 'conts']:
#     var_names[k] = [_v for _v in var_names[k] if _v not in to_remove[k]]
# var_names

In [ ]:
%%time
procs = [Categorify, FillMissing, Normalize]
df_rep = train_predict(df.copy(), var_names, model, model_params=model_params, 
                       n_rep=n_rep, n_samples_train=n_samples_train,
                       n_samples_valid=n_samples_valid, procs=procs,
                       split_params=split_params)

In [ ]:
df_rep['rmse loss'].describe()

In [ ]:
px.box(df_rep, y='rmse loss', range_y=(0, 2.5))

Baseline model = RandomForest with 20 estimators and sqrt features, training over 100k samples and predicting over 1k

<table>
    <tr>
        <th>input</th>
        <th>model</th>
        <th>rmse loss</th>
        <th>time [s/it]</th>
    </tr>
    <tr>
        <td>meter and building id only</td>
        <td>random forest</td>
        <td>1.2 - 1.21</td>
        <td>10.2</td>
    </tr>
    <tr>
        <td>using dep_var stats</td>
        <td>random forest</td>
        <td>1.16 - 1.18</td>
        <td>17.3</td>
    </tr>
    <tr>
        <td>using time stats</td>
        <td>random forest</td>
        <td>1.2 - 1.21</td>
        <td>13.2 - 13.7</td>
    </tr>
    <tr>
        <td>using building info</td>
        <td>random forest</td>
        <td>1.19</td>
        <td>17 - 18</td>
    </tr>
    <tr>
        <td>using weather (+ building) info</td>
        <td>random forest</td>
        <td>1.13 - 1.139</td>
        <td>14.6 - 15</td>
    </tr>
    <tr>
        <td>using all above</td>
        <td>random forest</td>
        <td>1.19 - 1.21</td>
        <td>20 - 26</td>
    </tr>
    <tr>
        <td>removing leading 0s in `dep_var`</td>
        <td>random forest</td>
        <td>.36 - .37</td>
        <td>4</td>
    </tr>
    <tr>
        <td>removing trailing 0s in `dep_var`</td>
        <td>random forest</td>
        <td>1.2</td>
        <td>4</td>
    </tr>
    <tr>
        <td>removing empty weeks before the first full week</td>
        <td>random forest</td>
        <td>1.16</td>
        <td>4</td>
    </tr>
    <tr>
        <td>meter only</td>
        <td>linear model</td>
        <td>2.2</td>
        <td>5</td>
    </tr>
    <tr>
        <td>meter + hour</td>
        <td>linear model</td>
        <td>2.1</td>
        <td>5</td>
    </tr>
    <tr>
        <td>meter reading stats only (meter, building_id, hour)</td>
        <td>linear model</td>
        <td>1.23 - 1.24 / 1.68 - 1.7</td>
        <td>5</td>
    </tr>
    <tr>
        <td>meter + meter reading stats (meter, building_id, hour)</td>
        <td>linear model</td>
        <td>1.51 - 1.52</td>
        <td>5</td>
    </tr>
    <tr>
        <td>meter reading stats (meter, building_id, hour)</td>
        <td>random forest</td>
        <td>0.58 - 0.6</td>
        <td>5</td>
    </tr>
    <tr>
        <td>meter + meter reading stats (meter, building_id, hour)</td>
        <td>random forest</td>
        <td>1.21 - 1.22</td>
        <td>5</td>
    </tr>
</table>

## Comparing `dep_var` distributions

In [ ]:
%%time
splits = preprocessing.split_dataset(df, **split_params)

In [ ]:
%%time
to = get_tabular_object(df, var_names, splits=splits)

In [ ]:
n_samples_train = 100_000
n_samples_valid = 100_000

In [ ]:
%%time
params = {'n_estimators': 100, 'max_features': 'sqrt',
          'min_samples_leaf':5}
model = ensemble.RandomForestRegressor
m = model(**params)

In [ ]:
%%time
_X = to.train.xs.sample(n_samples_train, replace=True).values
_y = to.train.ys.sample(n_samples_train, replace=True).values.ravel()
m.fit(_X, _y)

In [ ]:
%%time
_X = to.valid.xs.sample(n_samples_valid, replace=True).values
_y = to.valid.ys.sample(n_samples_valid, replace=True).values.ravel()
pred = m.predict(_X)

In [ ]:
#hide
assert np.isfinite(_y).all() and np.isfinite(pred).all()
assert _y.shape == pred.shape

In [ ]:
#export
def hist_plot_preds(y0:np.ndarray, y1:np.ndarray,
                    label0:str='y0', label1:str='y1'):
    res = pd.concat(
        (
            pd.DataFrame({
                'y': y0,
                'set': [label0] * len(y0)
            }),
            pd.DataFrame({
                'y':y1,
                'set': [label1] * len(y1)
            })
        ),
        ignore_index=True
    )

    return px.histogram(res, x='y', color='set', marginal='box',
                        barmode='overlay', histnorm='probability density')

In [ ]:
hist_plot_preds(_y, pred, label0='truth (valid)', label1='prediction (valid)')

## Inspecting confidently wrong predictions

In [ ]:
#export
class BoldlyWrongTimeseries:
    def __init__(self, xs, y_true, y_pred, info:pd.DataFrame=None):
        if info is None:
            self.df = xs.loc[:,['meter', 'building_id', 'timestamp']].copy()
        else:
            assert all([v in info.columns.values for v in ['meter', 'building_id', 'timestamp']])
            self.df = xs.join(info)

        for col in ['meter', 'building_id']:
            self.df[col] = self.df[col].astype('category')
            self.df[col].cat.set_categories(sorted(self.df[col].unique()),
                                            ordered=True, inplace=True)

        self.df['y_true'] = y_true
        self.df['y_pred'] = y_pred
        self.compute_misses()

    def compute_misses(self):
        fun = lambda x: np.sqrt(np.mean(x**2))
        self.miss = (self.df.assign(difference=lambda x: x['y_pred']-x['y_true'])
                     .groupby(['building_id', 'meter'])
                     .agg(loss=pd.NamedAgg(column='difference', aggfunc=fun))
                     .dropna()
                     .sort_values('loss'))

In [ ]:
%%time
_X = to.valid.xs 
_y = to.valid.ys.values.ravel() 
pred = m.predict(to.valid.xs.values)

In [ ]:
assert _y.shape == pred.shape

In [ ]:
%%time
bwt = BoldlyWrongTimeseries(to.valid.xs.join(df.loc[:,['building_id', 'meter','timestamp']], lsuffix='to_'), 
                            _y, pred)

In [ ]:
#hide
if loading.N_TRAIN is None: assert len(bwt.miss) == 2380

Adding plotting capability based on the loss or meter/building id

In [ ]:
#export
@patch
def plot_boldly_wrong(self:BoldlyWrongTimeseries,
                      nth_last:int=None,
                      meter:int=None, bid:int=None):

    assert (meter is not None and bid is not None) or (nth_last is not None)

    if nth_last is not None:
        ix = self.miss.iloc[[nth_last],:]
        meter = ix.index[0][1]
        bid = ix.index[0][0]
        loss = ix["loss"].values[0]
    else:
        ix = self.miss.xs((bid,meter))
        loss = ix.values[0]


    df_plot = self.df.loc[(self.df['meter']==int(meter)) & (self.df['building_id']==int(bid))]
    df_plot = pd.concat((
        df_plot[['timestamp', 'y_true']].rename(columns={'y_true':'y'}).assign(label='true'),
        df_plot[['timestamp', 'y_pred']].rename(columns={'y_pred':'y'}).assign(label='pred')),
        ignore_index=True
    )
    return df_plot.plot(kind='scatter', x='timestamp',
                        y='y', color='label', opacity=.4,
                        title=f'pos {nth_last}: meter = {meter}, building_id = {bid}<br>loss = {loss:.3f}')

In [ ]:
bwt.plot_boldly_wrong(nth_last=-1)

In [ ]:
bwt.plot_boldly_wrong(meter=2, bid=1099)

Adding widgets for interactive exploration

In [ ]:
#export
@patch
def init_widgets(self:BoldlyWrongTimeseries):
    self.int_txt_loss = widgets.IntText(min=-len(self.miss), max=len(self.miss),
                                        description='Position', value=-1)
    self.int_txt_meter = widgets.IntText(min=self.df['meter'].min(), max=self.df['meter'].max(),
                                         description='Meter')
    self.int_txt_bid = widgets.IntText(min=self.df['building_id'].min(), max=self.df['building_id'].max(),
                                       description='building id')
    self.run_btn = widgets.Button(description='plot')
    self.switch_btn = widgets.Checkbox(description='Loss-based', value=True)
    self.run_btn.on_click(self.click_boldly_wrong)
    self.out_wdg = widgets.Output()

@patch
def run_boldly(self:BoldlyWrongTimeseries):
    if not hasattr(self, 'switch_btn'):
        self.init_widgets()
    return widgets.VBox([self.switch_btn, self.int_txt_loss,
                         self.int_txt_meter, self.int_txt_bid,
                         self.run_btn, self.out_wdg])

@patch
def click_boldly_wrong(self:BoldlyWrongTimeseries, change):
    self.out_wdg.clear_output()
    nth_last = None if self.switch_btn.value == False else self.int_txt_loss.value
    meter = None if self.switch_btn.value == True else self.int_txt_meter.value
    bid = None if self.switch_btn.value == True else self.int_txt_bid.value
    with self.out_wdg:
        print(f'nth_last {nth_last} meter {meter} bid {bid}')
        try:
            self.plot_boldly_wrong(nth_last=nth_last, meter=meter, bid=bid).show()
        except:
            raise ValueError(f'nth_last {nth_last} meter {meter} bid {bid} not a valid combination! Likely due to missing meter/bid combination')

In [ ]:
bwt.run_boldly()

In [ ]:
from nbdev.export import *
notebook2script()